## 加载数据

In [2]:
import numpy as np
import scipy.sparse as sp
import torch

In [4]:
content_path = "./data/cora/cora.content"
idx_features_labels = np.genfromtxt(content_path, dtype=np.dtype(str))

In [5]:
features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
features.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
def encode_onehot(labels):
    # 获取类别数
    classes = set(labels)
    # 生成一个对角矩阵，并生成标签和编码向量对应的字典
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [11]:
labels = encode_onehot(idx_features_labels[:, -1])

### 建立图结构

In [12]:
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)}
edges_unordered = np.genfromtxt("./data/cora/cora.cites", dtype=np.int32)
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

# 建立对称矩阵
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

In [14]:
adj.toarray().shape

(2708, 2708)

### 特征正则化和邻接矩阵正则化

In [15]:
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [16]:
features = normalize_features(features)
adj = normalize_adj(adj + sp.eye(adj.shape[0]))

### 划分训练集、验证集和测试集

In [20]:
np.where(labels)

(array([   0,    1,    2, ..., 2705, 2706, 2707]),
 array([3, 4, 0, ..., 2, 1, 3]))

In [21]:
idx_train = range(140)
idx_val = range(200, 500)
idx_test = range(500, 1500)

adj = torch.FloatTensor(np.array(adj.todense()))
features = torch.FloatTensor(np.array(features.todense()))
labels = torch.LongTensor(np.where(labels)[1])

idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)

## GAT 模型定义

1. 自定义注意力层；
2. 结合注意力层定义图注意力神经网络模型；

### 定义注意力层

In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class GraphAttentionLayer(nn.Module):
    """
    GAT layer
    """
    
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
        self.leakyrelu = nn.LeakyReLU(self.alpha)
    
    def forward(self, input, adj):
        h = torcn.mm(input, self.W)
        N = h.size()[0]
        a_input = torch.cat([h.repeat(1, N).])

In [27]:
W = torch.zeros(size=(features.size()[1], 128))
nn.init.xavier_uniform_(W, gain=1.414)
W

tensor([[-0.0300, -0.0253, -0.0741,  ..., -0.0199, -0.0513, -0.0686],
        [ 0.0845,  0.0470,  0.0734,  ...,  0.0197, -0.0161, -0.0497],
        [-0.0503,  0.0409, -0.0848,  ..., -0.0310,  0.0731, -0.0131],
        ...,
        [ 0.0464, -0.0514,  0.0559,  ..., -0.0366, -0.0549, -0.0839],
        [-0.0044, -0.0607, -0.0681,  ...,  0.0745,  0.0571, -0.0115],
        [-0.0340,  0.0791,  0.0686,  ..., -0.0747,  0.0764,  0.0582]])

In [38]:
h = torch.mm(features, W)
N = h.size()[0]

a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * 128)
a_input.size()

torch.Size([2708, 2708, 256])

In [39]:
leakyrelu = nn.LeakyReLU(0.2)
leakyrelu

LeakyReLU(negative_slope=0.2)

In [41]:
a = torch.zeros(size=(2*128, 1))
nn.init.xavier_uniform_(a, gain=1.414)

tensor([[-1.2024e-01],
        [-1.1419e-01],
        [-1.7044e-01],
        [ 1.7360e-01],
        [ 1.7264e-02],
        [ 7.2557e-02],
        [ 4.5110e-02],
        [-4.2041e-02],
        [ 1.5762e-01],
        [ 1.7810e-01],
        [-2.0461e-01],
        [ 1.1418e-01],
        [ 4.2414e-02],
        [ 1.4858e-01],
        [-2.9029e-02],
        [ 1.9405e-01],
        [ 2.0378e-01],
        [-1.9727e-01],
        [ 1.9980e-01],
        [ 1.8286e-01],
        [ 6.9735e-02],
        [ 4.1901e-02],
        [-4.4165e-02],
        [-9.8491e-02],
        [-3.3567e-02],
        [-2.1180e-01],
        [-8.7822e-02],
        [ 9.9121e-02],
        [ 4.1836e-02],
        [-1.0435e-01],
        [-1.9050e-02],
        [-1.3781e-01],
        [-1.8584e-01],
        [ 6.2340e-02],
        [ 8.6222e-02],
        [-1.9500e-01],
        [-6.2649e-02],
        [-2.0984e-01],
        [-1.9376e-01],
        [-8.9739e-02],
        [ 2.1301e-01],
        [ 1.2192e-01],
        [-1.4777e-01],
        [ 1

In [47]:
e = leakyrelu(torch.matmul(a_input, a)).squeeze(2)
e.size()

torch.Size([2708, 2708])

In [49]:
e

tensor([[ 2.4185e-02, -3.4654e-03,  1.1915e-05,  ..., -9.2427e-03,
         -1.7768e-03, -1.4830e-03],
        [ 4.1635e-02,  1.2280e-04,  1.7462e-02,  ..., -5.7527e-03,
          8.5660e-03,  1.0035e-02],
        [ 8.9432e-03, -6.5138e-03, -3.0460e-03,  ..., -1.2291e-02,
         -4.8252e-03, -4.5314e-03],
        ...,
        [-8.3743e-04, -9.1399e-03, -5.6721e-03,  ..., -1.4917e-02,
         -7.4513e-03, -7.1575e-03],
        [ 5.1216e-02,  9.7038e-03,  2.7043e-02,  ..., -3.8365e-03,
          1.8147e-02,  1.9616e-02],
        [ 2.2734e-02, -3.7557e-03, -2.8790e-04,  ..., -9.5330e-03,
         -2.0671e-03, -1.7733e-03]])

In [50]:
zero_vec = -9e15 * torch.ones_like(e)
attention = torch.where(adj > 0, e, zero_vec)
attention = F.softmax(attention, dim=1)
attention = F.dropout(attention, 0.6, training=)

tensor([[0.1703, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5012, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2006,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.1991, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1995, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2470]])

In [ ]:
content_path = "./data/cora/cora.content"
idx_features_labels = np.genfromtxt(content_path, dtype=np.dtype(str))